In [19]:
import urllib.request
import json
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [2]:
def calculate_hours_from_date(start_date_str):
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M')
    now = datetime.now()
    hours = int((now - start_date).total_seconds() / 3600)
    return hours

start_date = "2022-11-02 00:00"
hours_limit = calculate_hours_from_date(start_date)
print(f"Hours from {start_date} until now: {hours_limit}")

Hours from 2022-11-02 00:00 until now: 21829


In [17]:
api_key = "2bb948aa-5b3a-414f-9ce4-3c3b34875f8d"

parameters = ["mean_temp", "mean_wind_speed", "mean_radiation", "mean_wind_dir"]

all_weather_data = []

In [18]:
for param in parameters:
    url = f"https://dmigw.govcloud.dk/v2/climateData/collections/countryValue/items?api-key={api_key}&parameterId={param}&limit={hours_limit}&timeResolution=hour"
    
    # Fetch data from API
    with urllib.request.urlopen(url) as fh:
        data = json.load(fh)
    
    # Extract relevant fields
    records = []
    for feature in data.get("features", []):
        properties = feature.get("properties", {})
        records.append({
            "Datetime": properties.get("from"),  # Timestamp
            "Parameter": param,
            "Value": properties.get("value")
        })
    
    df = pd.DataFrame(records)
    all_weather_data.append(df)

final_df = pd.concat(all_weather_data)
print("Checking for duplicates before pivoting:")
print(final_df[final_df.duplicated(subset=['Datetime', 'Parameter'], keep=False)])
final_df = final_df.pivot_table(index="Datetime", columns="Parameter", values="Value", aggfunc='mean').reset_index()

final_df.to_csv("../data/DMI_data.csv", index=False)

Checking for duplicates before pivoting:
Empty DataFrame
Columns: [Datetime, Parameter, Value]
Index: []


In [20]:
file_path = os.path.join('..', 'data', 'DMI_data.csv') # Adjust path if needed
wind_dir_col = 'mean_wind_dir'

# Read the CSV
df = pd.read_csv(file_path)

# Convert to sin/cos if the column exists
if wind_dir_col in df.columns:
    # Ensure numeric, fill potential NaNs with 0
    df[wind_dir_col] = pd.to_numeric(df[wind_dir_col], errors='coerce').fillna(0)

    # Convert degrees to radians and calculate sin/cos
    wind_dir_rad = np.radians(df[wind_dir_col])
    df['wind_dir_sin'] = np.sin(wind_dir_rad)
    df['wind_dir_cos'] = np.cos(wind_dir_rad)

    # Drop the original column
    df = df.drop(columns=[wind_dir_col])

    # Save back to CSV, overwriting the original
    df.to_csv(file_path, index=False)